In [1]:
import os
os.environ["THEANORC"] = "~/.theanorc.cuda"

%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L

Using cuDNN version 5105 on context None
Preallocating 598/1994 Mb (0.300000) on cuda
Mapped name None to device cuda: GeForce GTX 660 (0000:01:00.0)


In [2]:
%aimport QANet

floatX == float32
device == cuda


In [3]:
%aimport QANet_old

floatX == float32
device == cuda


In [2]:
import sys
sys.path.insert(0, '../../masters_thesis_old/SQuAD/')

from QANet import QANet as QANet_orig

('floatX ==', 'float32')
('device ==', 'cuda')


In [3]:
glove_path = '/pio/data/data/glove_vec/6B/glove/'
glove_embs = np.load(glove_path + 'glove.6B.300d.npy')
voc_size = glove_embs.shape[0]

# Weryfikacja modeli

In [4]:
%%time
qa_net = QANet.QANet(voc_size=voc_size,
                     emb_init=glove_embs,
                     train_inds=[],
                     emb_dropout=True,
                     skip_train_fn=True,
                     prefetch_word_embs=True)

Building the model...
Using dropout.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 30.1 s, sys: 392 ms, total: 30.5 s
Wall time: 30.9 s


In [6]:
%%time
qa_net_old = QANet_old.QANet(voc_size=voc_size,
                             emb_init=glove_embs,
                             train_inds=[],
                             skip_train_fn=True,
                             prefetch_word_embs=True)

Building the model...
Compiling theano functions:
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 26.9 s, sys: 456 ms, total: 27.3 s
Wall time: 30.5 s


In [6]:
%%time
qa_net_orig = QANet_orig(voc_size=voc_size,
                         alphabet_size=128,
                         emb_size=300,
                         emb_char_size=20,
                         num_emb_char_filters=200,
                         rec_size=300,
                         emb_init=glove_embs,
                         train_inds=[])

Building the model...
Compiling theano functions:
    train_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 3min 28s, sys: 4.78 s, total: 3min 33s
Wall time: 5min 1s


In [10]:
qa_net.load_params('trained_models/glove_vocab/wiki_negative/charemb_trainNAW_dropout_memory_crashed/charemb_trainNAW_dropout_ep5.npz', glove_embs)

In [7]:
qa_net.load_params('../../qa_nips/data/charemb_glove_fixed_dropout_ep13.npz', glove_embs)

In [ ]:
qa_net.load_params('trained_models/glove6B/charemb_fixed_dropout2/charemb_fixed_dropout2_ep3.npz', glove_embs)

In [11]:
qa_net_old.load_params('trained_models/glove6B/charemb_fixed_dropout2/charemb_fixed_dropout2_ep3.npz', glove_embs)

In [8]:
qa_net_orig.load_params('trained_models/glove6B/charemb_fixed_dropout2/charemb_fixed_dropout2_ep3.npz', glove_embs)

In [ ]:
%%time
qa_net._calc_dev_f1(0) # float32, intermediate

Calculating validation f1...
Done 2500 examples
Done 5000 examples


In [9]:
%%time
qa_net._calc_dev_f1(0) # float32, intermediate, broken evaluation (nondeterministic dropout!)

Calculating validation f1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  59.9639778539
CPU times: user 1min 56s, sys: 39.7 s, total: 2min 36s
Wall time: 2min 38s


59.963977853901525

In [8]:
%%time
qa_net._calc_dev_f1(0) # float64 eye, intermediate

Calculating validation f1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  62.6536004484
CPU times: user 14min 27s, sys: 18.2 s, total: 14min 46s
Wall time: 2min 40s


62.653600448351575

In [9]:
%%time
qa_net_old._calc_dev_f1(0) # float32, no intermediate

Calculating validation f1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  62.5606276977
CPU times: user 1min 57s, sys: 2.66 s, total: 1min 59s
Wall time: 2min 2s


62.560627697673056

In [9]:
%%time
qa_net_old._calc_dev_f1(0) # float64 eye, no intermediate

Calculating validation f1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  62.6536004484
CPU times: user 18min 54s, sys: 23.5 s, total: 19min 18s
Wall time: 3min 23s


62.653600448351575

In [9]:
%%time
qa_net_orig._calc_dev_f1(0)

Calculating validation f1...
Done 1000 examples
Done 2000 examples
Done 3000 examples
Done 4000 examples
Done 5000 examples
Done 6000 examples
Done 7000 examples
Done 8000 examples
Done 9000 examples
Done 10000 examples
Predictions done
F1:  62.6536004484
CPU times: user 18min 56s, sys: 24.3 s, total: 19min 20s
Wall time: 3min 27s


62.653600448351575

In [5]:
qa_net._calc_dev_f1(0)

Calculating validation f1...
Done 1000 examples
Done 2000 examples
Done 3000 examples
Done 4000 examples
Done 5000 examples
Done 6000 examples
Done 7000 examples
Done 8000 examples
Done 9000 examples
Done 10000 examples
Predictions done
F1:  69.5643803522


69.56438035215977

In [ ]:
'''
charemb_trainNAW_dropout_memory_crashed/charemb_trainNAW_dropout_ep5.npz
    Pos: F1: 68.12   Exact: 58.04
    Neg: F1: 60.17   Exact: 51.84
    
    Classification task:
        [[TN=9857  FP= 455]
         [FN=1275  TP=9295]]
    
charemb_glove_fixed_dropout_ep13.npz
    Pos: F1: 69.56   Exact: 59.33
'''